In [3]:
import fitz  # PyMuPDF
import faiss
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document
from langchain.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os
from langchain.vectorstores import Chroma 
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma 
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.chains import RetrievalQA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer
import torch

In [5]:
def setup_environment():
    import sys
    sys.path.append('C:\\gitworkspace\\aimldemo\\jupyterworkapce')
    import stratup_env_setup
    stratup_env_setup.set_env()

In [6]:
setup_environment()

Set env key: AWS_ACCESS_KEY_ID
Set env key: AWS_SECRET_ACCESS_KEY
Set env key: OPENAI_API_KEY
Set env key: HUGGINGFACEHUB_API_TOKEN
Set env key: LANGCHAIN_API_KEY


In [7]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [8]:
llm_mistral = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2")

C:\Users\jimmy\AppData\Local\Temp\ipykernel_17008\936897557.py:1: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm_mistral = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\jimmy\.cache\huggingface\token
Login successful


In [9]:
chunk_size=1000

In [10]:
pdf_file_path = "doc_inputs/Form_2287.pdf"
#pdf_file_path = "doc_inputs/John-NIV.pdf"
#pdf_file_path = 'doc_inputs/falcon-users-guide-2021-09.pdf'

In [11]:
#query = "In the event of default under the Third Party Loan, what will CDC or SBA may do?"
#query = 'what does this document say about Authority to Execute Agreement?'
#query = 'What miraculous sign can you show us to prove your authority to do all this?'
#query = 'What does this document say about waiver of Right to Indemnification by SBA or CDC.'
#query = 'where can I find the standard price for Falcon 9 and Falcon Heavy launch services?'
#query = 'SpaceX requires that customers verify the mass properties of their system through measurement before shipping it to the launch site.Is this true or false?'
query = 'What does this document say about Maturity and Balloon Payments?'
#query = 'Hello How are you?'

# Mistral, Chroma DB and PyPDFLoader

In [9]:
query

'What does this document say about Maturity and Balloon Payments?'

In [10]:
pdf_loader = PyPDFLoader(pdf_file_path)

In [11]:
documents = pdf_loader.load()

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=200)

In [13]:
chunks = text_splitter.split_documents(documents)

In [14]:
type(chunks[0])

langchain_core.documents.base.Document

In [15]:
embeddings = OpenAIEmbeddings()

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24816\2497576997.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [16]:
# Get embeddings for the query
query_embedding = embeddings.embed_query(query)

In [17]:
# Print top 3 similar chunks before persisting into DB
# Calculate the similarity for each chunk and store it along with the chunk
similarities = []
for i, chunk in enumerate(chunks):
    # Get embedding for the chunk
    chunk_embedding = embeddings.embed_query(chunk.page_content)
    
    # Calculate cosine similarity between query and chunk
    similarity = cosine_similarity([query_embedding], [chunk_embedding])[0][0]
    
    # Store similarity, chunk content, and metadata for sorting
    similarities.append((similarity, chunk.page_content, chunk.metadata))
    
# Sort the chunks based on similarity in descending order and get the top 3
top_chunks = sorted(similarities, key=lambda x: x[0], reverse=True)[:3]

# Print the top 3 chunks
for i, (similarity, content, metadata) in enumerate(top_chunks, start=1):
    print(f"Top Chunk {i}: Page number: {metadata['page']}")
    print(content)
    print(f"Cosine Similarity: {similarity * 100:.4f}")
    print("#" * 100)  # Optional separator for readability

Top Chunk 1: Page number: 2
exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third 
Party Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the 
Third Party Loan prior to maturity unless the Third Party L oan is in material  default .   
 
 e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at 
least 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the 
504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an 
overall loan maturity must be calculated , taking into account the amounts and maturities  of each 
loan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to 
SBA and approved at application or subsequently a pproved by SBA .  
 
 f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate
Cosine Similarity: 83.3185
########

In [18]:
vector_store = Chroma.from_documents(chunks, embeddings)

In [19]:
retriever=vector_store.as_retriever()

In [20]:
retrieved_docs = retriever.get_relevant_documents(query)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24816\3272410983.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [22]:
type(retrieved_docs[0])

langchain_core.documents.base.Document

In [23]:
unique_docs = set()

In [24]:
for doc in retrieved_docs:
    unique_docs.add(doc.page_content)  # Replace 'page_content' with a unique identifier if available   


In [25]:
print(f"Total Retrieved Documents: {len(retrieved_docs)}")
print(f"Unique Documents Retrieved: {len(unique_docs)}")

Total Retrieved Documents: 4
Unique Documents Retrieved: 4


In [26]:
# Print the documents and their cosine similarity with the query
for i, d in enumerate(retrieved_docs):
    # Get the embedding for the current document chunk
    doc_embedding = embeddings.embed_query(d.page_content)
    
    # Compute the cosine similarity between the query and the document
    similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
    
    # Print the document content and the similarity score
    print(f"Document {i + 1}:Page number: {d.metadata['page']}:  {d.page_content}")
    print(f"Cosine Similarity: {similarity*100:.4f}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

Document 1:Page number: 2:  exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third 
Party Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the 
Third Party Loan prior to maturity unless the Third Party L oan is in material  default .   
 
 e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at 
least 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the 
504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an 
overall loan maturity must be calculated , taking into account the amounts and maturities  of each 
loan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to 
SBA and approved at application or subsequently a pproved by SBA .  
 
 f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate
Cosine Similarity: 83.3185
########

In [27]:
for i, d in enumerate(retrieved_docs):
    print(f"Document {i + 1}: {d.page_content}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

Document 1: exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third 
Party Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the 
Third Party Loan prior to maturity unless the Third Party L oan is in material  default .   
 
 e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at 
least 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the 
504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an 
overall loan maturity must be calculated , taking into account the amounts and maturities  of each 
loan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to 
SBA and approved at application or subsequently a pproved by SBA .  
 
 f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate
###################################################

In [28]:
type(retrieved_docs)

list

In [29]:
retrieved_docs

[Document(metadata={'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third \nParty Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party L oan is in material  default .   \n \n e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .  \n \n f. Reasonable Interest Rate.   The Third Party Loan

In [30]:
retrieved_doc = retrieved_docs[0] 

In [31]:
retrieved_doc

Document(metadata={'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third \nParty Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party L oan is in material  default .   \n \n e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .  \n \n f. Reasonable Interest Rate.   The Third Party Loan 

In [32]:
retrieved_text = retrieved_doc.page_content

In [33]:
retrieved_metadata = retrieved_doc.metadata

In [34]:
retrieved_text

'exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third \nParty Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party L oan is in material  default .   \n \n e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .  \n \n f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate'

In [35]:
retrieved_metadata

{'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}

In [36]:
embeddings = OpenAIEmbeddings()
llm_gpt = ChatOpenAI(model='gpt-4', temperature=1)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24816\2899348281.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_gpt = ChatOpenAI(model='gpt-4', temperature=1)


* Use ContextualCompressionRetriever

In [37]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [38]:
compressor = LLMChainExtractor.from_llm(llm_gpt)

In [39]:
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [40]:
docs_compressed = compression_retriever.get_relevant_documents(query)

In [41]:
docs_compressed

[Document(metadata={'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .')]

In [42]:
docs_compressed[0]

Document(metadata={'page': 2, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .')

In [43]:
system_prompt = (
    "You are an assistant for question-answering tasks. Answer the following question based only on the context provided. Do not use any external knowledge."
     "\n\n"
    "{context}"
)

In [44]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [45]:
question_answer_chain = create_stuff_documents_chain(llm_gpt, prompt)

In [46]:
inputs = {
    "context": [retrieved_docs[0]],  # use only the first elemnt from retrieved_docs
    "input": query  # Use the query as the input for the human message
}

In [47]:
response = question_answer_chain.invoke(inputs)

In [48]:
response

'The document states that the Third Party Loan must have a term of at least 7 years when the 504 loan is for a term of 10 years, or a term of at least 10 years when the 504 loan is for 20 or 25 years. If the Third Party Lender has made more than one loan, then an overall loan maturity must be calculated, considering the amounts and maturities of each loan. Any balloon payment for the Third Party Loan must be clearly identified and disclosed to the SBA and approved at the time of application or subsequently approved by the SBA.'

In [49]:
type(response)

str

In [50]:
input_text = f"""
Context: {retrieved_text}

Instruction: Answer the following question based only on the context provided. Do not use any external knowledge.

Question: {query}

Answer:
"""

In [51]:
input_text

'\nContext: exercise any cross -default, "deem at -risk," or any other provisions in documents evidencing the Third \nParty Loan  or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party L oan is in material  default .   \n \n e. Matur ity and Balloon Payments.   The Third Party Loan  must ha ve a term of at \nleast 7 years ( when the 504 loan is for a term of 10 years), or a term of at least 10 years (when the \n504 loan is for 20 or 25 years ).  If the Third Party Lender has made more than one loan, then  an \noverall loan maturity must be calculated , taking into account the amounts and maturities  of each \nloan.  Any balloon payment  for the Third Party Loan must be clearly  identifie d and disclosed to \nSBA and approved at application or subsequently a pproved by SBA .  \n \n f. Reasonable Interest Rate.   The Third Party Loan has a reasonable interest rate\n\nInstruction: Answer the following que

In [52]:

response_gpt = llm_gpt.predict(input_text)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24816\2746106821.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_gpt = llm_gpt.predict(input_text)


In [53]:
response_gpt

'The document states that the Third Party Loan must have a term of at least 7 years if the 504 loan is for a term of 10 years, or a term of at least 10 years if the 504 loan is for 20 or 25 years. If the Third Party Lender has made more than one loan, an overall loan maturity must be calculated considering the amounts and maturities of each loan. Any balloon payment for the Third Party Loan must be clearly identified, disclosed to SBA, and be approved at the application stage or subsequently approved by SBA.'

In [54]:
#chain_mistral = RetrievalQA.from_chain_type(llm_mistral, retriever=retriever )
#response_mistral = chain_mistral.run(input_text)
response_mistral = llm_mistral.predict(input_text)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24816\3110740755.py:3: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_mistral = llm_mistral.predict(input_text)


In [55]:
response_mistral

'This document outlines requirements related to Maturity and Balloon Payments for a Third Party Loan. The loan must have a term of at least 7 years when the 504 loan is for a term of 10 years, or a term of at least 10 years when the 504 loan is for 20 or 25 years. If the Third Party Lender has made more than one loan, then an overall loan maturity must be calculated, taking into account the amounts and maturities of each loan. Any balloon payment for the Third Party Loan must be clearly identified and disclosed to SBA and approved at application or subsequently approved by SBA.'

In [56]:
# Highlight using cosine similarity
# Step 1: Extract content to be highlighted from the first retrieved document
content_to_highlight = retrieved_text

# Step 2: Open the original PDF
pdf_document = fitz.open(pdf_file_path)

# Step 3: Initialize variables for tracking the best match
best_page = None
best_match_rects = None
highest_similarity = 0
best_page_number = None  # Initialize best_page_number her

# Step 4: Iterate through each page in the PDF
for page_num in range(len(pdf_document)):
    page = pdf_document[page_num]
    page_text = page.get_text()

    # Step 5: Split the page text into smaller chunks
    page_chunks = [page_text[i:i+chunk_size] for i in range(0, len(page_text), chunk_size)]

    # Step 6: Perform text similarity search using TF-IDF and cosine similarity
    vectorizer = TfidfVectorizer().fit_transform([content_to_highlight] + page_chunks)
    similarities = cosine_similarity(vectorizer[0:1], vectorizer[1:]).flatten()

    # Step 7: Find the chunk with the highest similarity score
    max_sim_index = similarities.argmax()
    max_sim = similarities[max_sim_index]

    # Step 8: Update the best match if this page has a higher similarity score
    if max_sim > highest_similarity:
        highest_similarity = max_sim
        best_page = page
        best_chunk = page_chunks[max_sim_index]
        best_page_number = page_num + 1  # Store the 1-based page number

        # Locate the matching text in the page
        best_match_rects = best_page.search_for(best_chunk)

In [57]:
# Step 9: Highlight the best match
if best_page and best_match_rects:
    for rect in best_match_rects:
        best_page.add_highlight_annot(rect)

In [58]:
# Step 10: Print the best page number
if best_page_number is not None:
    print(f"The best match was found on page number: {best_page_number}")

The best match was found on page number: 3


In [59]:
# Step 10: Save the modified PDF
highlighted_pdf_path = "doc_outputs/highlighted_file_chroma.pdf"
pdf_document.save(highlighted_pdf_path)

In [60]:
# Close the PDF document
pdf_document.close()

* Query for meta data

In [61]:
metadata_field_info = [
    AttributeInfo(
        name="page",
        description="The page number within teh document.Starting with 0",
        type="integer",
    ),
    AttributeInfo(
        name="source",
        description="The name of the document as a file name",
        type="string",
    ),    
]

In [62]:
document_content_description = "Content of a specific page"

In [63]:
retriever = SelfQueryRetriever.from_llm(
    llm_gpt,
    vector_store,
    document_content_description,
    metadata_field_info,
)

In [64]:
retriever.invoke("I want the page number 1")

[Document(metadata={'page': 1, 'source': 'doc_inputs/Form_2287.pdf'}, page_content='behalf of SBA, an agency in the United States Government, except that SBA accepts no liability or \nresponsibility for any wrongful act or omission by CDC.  Third Part y Lender further acknowledges that any \nfalse statements to CDC can be considered false statement s to the federal government under 18 U.S.C. § 1001, \nand may subject the Third Party Lender to criminal penalties , and that CDC and SBA are relying upon the \ninfor mation submitted by the Third Party Lender.  \n \n4. Waiver of Provision Not to Encumber Common Collateral.    Third Party Lender waives its rights to \nenforce, as against CDC and SBA, any provisions in its documents that prohibit Borrower from further \nencumb ering the Common Collateral or which restrict Borrower’s ability to assign its lease on, or rents, \nincome or profits from, the Common Collateral . \n \n5. Compliance with 504 Loan Program Requirements.   Third Party L

# Chroma storeage using local BERT Embedding <span style="color:red;">(Embedding result is very poor)</span>

In [65]:
import uuid 

In [66]:
query = """Subordination of 504 Loan.  CDC agrees to make the 504 Loan to the Borrower , subject to SBA’s 
approval , and accept a junior and subordinate lien position in the Com mon Collateral upon the condition s that 
Third Party Lender executes this Agreement and disburses the  Third Party Loan  according to the terms 
represented to CDC and SBA"""

In [67]:
pdf_loader = PyPDFLoader(pdf_file_path)

In [68]:
documents = pdf_loader.load()

In [69]:
local_directory = './bert_downloads'

In [70]:
# Step 2: Split the documents into chunks
chunk_size = 1000  # Adjust this size as needed
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [71]:
for i, chunk in enumerate(chunks):
    # Get the chunk content
    content = chunk.page_content
    
    # Get the metadata (e.g., page number)
    metadata = chunk.metadata
    page_number = metadata.get('page', None)
    
    # Print the chunk content and page number
    print(f"Chunk {i + 1}:")
    print(f"Page Number: {page_number+1}")
    print(f"Content: {content}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

Chunk 1:
Page Number: 1
Content: SBA F orm 2287 ( 04-18)            1 
 Previous Editions Obsolete    
 
 
 
THIRD PARTY LENDER AGREEMENT  
 
 
 THIS THIRD PARTY LENDER AGREEMENT ( “Agreement ") is dated this ______ day 
of______________, 20____, by and between ________________________________________,  
(“Third Party Lender ”) whose address is ____________________________________________, 
and________________________________________________________, ( “CDC ”) whose address is 
_________________________________________________________________ . 
 
RECITALS  
 
1.  The Third Party Lender and CDC will provide separate loans to th e Borrower and Operating Company, if 
any (collectively “Borrower”) , according to the terms  in the Authorization for Debenture Guarantee (SBA 
504 Loan), as amended ( “Authorization” ).  The Third Party Lender will provide term financing ( “Third 
Party Loan ”), and t he CDC will provide a loan ( “504 Loan” ) funded by a debenture issued by the CDC
###########

In [72]:
# Step 3: Load the BERT model and tokenizer from the local directory
tokenizer = AutoTokenizer.from_pretrained(local_directory, local_files_only=True)
model = AutoModel.from_pretrained(local_directory, local_files_only=True)

C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [73]:
# Function to get embeddings using BERT
def get_embedding(text):
    # Tokenize and encode the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the last hidden state as the embedding
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

In [74]:
# Step 4: Create embeddings for the chunks
texts = [chunk.page_content for chunk in chunks]
embeddings = [get_embedding(text).tolist() for text in texts]  # Convert numpy arrays to lists
ids = [str(uuid.uuid4()) for _ in texts]  # Generate unique IDs for each chunk

In [75]:
# Step 5: Initialize the Chroma vector store with a specified collection name and persist directory
vector_store = Chroma(collection_name="my_collection", persist_directory="chroma_db")

C:\Users\jimmy\AppData\Local\Temp\ipykernel_24816\2736092487.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(collection_name="my_collection", persist_directory="chroma_db")


In [76]:
embeddings

[[0.22743114829063416,
  0.6425228118896484,
  1.0813482999801636,
  -0.28447896242141724,
  0.6070812940597534,
  -0.2359061986207962,
  0.3323182463645935,
  0.12552814185619354,
  -0.10325668007135391,
  -0.17580607533454895,
  -0.10009210556745529,
  -0.28938713669776917,
  -0.35670211911201477,
  0.21508248150348663,
  0.31661906838417053,
  0.9760352969169617,
  0.7737584710121155,
  -0.2038004845380783,
  -0.2388950139284134,
  -0.003937471657991409,
  0.7731446027755737,
  -0.23778438568115234,
  0.8418214917182922,
  0.608092188835144,
  0.6978991031646729,
  0.15514923632144928,
  0.36070045828819275,
  0.07094257324934006,
  -0.8867992162704468,
  -0.31664085388183594,
  0.6717641353607178,
  -0.4162510633468628,
  0.16482330858707428,
  0.13199852406978607,
  0.7271205186843872,
  0.44456180930137634,
  -0.1663365215063095,
  -0.8231634497642517,
  0.1794586330652237,
  0.1438300609588623,
  -0.38201019167900085,
  -0.5818694233894348,
  0.12436055392026901,
  0.39469844102

In [77]:
# Step 6: Manually add the texts and embeddings to the vector store using upsert
vector_store._collection.upsert(
    documents=texts,
    embeddings=embeddings,
    metadatas=[chunk.metadata for chunk in chunks],
    ids=ids,  # Provide the generated unique IDs
)

In [78]:
# Step 7: Retrieve relevant documents based on the query manually
query_embedding = get_embedding(query).tolist()

In [81]:
query_embedding

[-0.38554847240448,
 -0.20317669212818146,
 0.4964841306209564,
 -0.07433915883302689,
 0.6229079365730286,
 -0.1414656937122345,
 -0.3256378471851349,
 0.16064485907554626,
 0.058194804936647415,
 0.084146648645401,
 -0.21019954979419708,
 -0.24928133189678192,
 0.0073090665973722935,
 0.26599887013435364,
 -0.2354584038257599,
 0.21868567168712616,
 0.5939397215843201,
 -0.022636689245700836,
 -0.018168555572628975,
 0.019777709618210793,
 0.07072792947292328,
 -0.15530253946781158,
 0.10021622478961945,
 0.3187420666217804,
 0.32722577452659607,
 0.3039550185203552,
 0.20691286027431488,
 0.13065442442893982,
 -0.318252295255661,
 -0.1530812680721283,
 0.8124557137489319,
 0.2412727177143097,
 -0.1720278412103653,
 -0.3961833119392395,
 0.16632574796676636,
 0.36874884366989136,
 0.08471883088350296,
 -0.36814019083976746,
 -0.6037116646766663,
 -0.03515980392694473,
 -0.38653409481048584,
 0.036229491233825684,
 -0.17446397244930267,
 0.10390126705169678,
 -0.2060590237379074,
 -0.

In [82]:
top_k=4

In [83]:
# Function to compute cosine similarity and retrieve top matches
# Compute similarities for all chunks
similarities = []
for i, (embedding, text) in enumerate(zip(embeddings, texts)):
    similarity = cosine_similarity([query_embedding], [embedding])[0][0]
    similarities.append((similarity, text))

# Sort by similarity in descending order and select top_k chunks
top_chunks = sorted(similarities, key=lambda x: x[0], reverse=True)[:top_k]

# Print the top chunks with their similarity scores
for i, (similarity, content) in enumerate(top_chunks, start=1):
    print(f"Top Chunk {i}:")
    print(content)
    print(f"Cosine Similarity: {similarity * 100:.4f}")
    print("#" * 100)  # Separator for readability


Top Chunk 1:
to the 504 Loan.  
 
2. Subordination of 504 Loan.  CDC agrees to make the 504 Loan to the Borrower , subject to SBA’s 
approval , and accept a junior and subordinate lien position in the Com mon Collateral upon the condition s that 
Third Party Lender executes this Agreement and disburses the  Third Party Loan  according to the terms 
represented to CDC and SBA .   
 
3.   Accurate Information.  The Third Party Lender warrants and represents that all i nformation provided 
by the Third Party Lender  to CDC, including, without limitation, all information regarding the Borrower’s 
financial condition, is accurate to the best of its knowledge and that Third Party Lender has not withheld any 
material informatio n.  Third Party Lender acknowledges that for purpose of this transaction, CDC is acting on 
behalf of SBA, an agency in the United States Government, except that SBA accepts no liability or
Cosine Similarity: 95.5625
###################################################

In [84]:
# Step 7: Retrieve the top 3 relevant documents from Chroma based on the query
top_k = 3  # Number of top documents to retrieve
retrieved_docs = vector_store._collection.query(
    query_embeddings=[query_embedding],
    n_results=top_k,
    include=['documents', 'embeddings', 'metadatas']
)

In [85]:
unique_docs = set() 

In [86]:
# Assuming each retrieved document has an ID or unique content
for doc in retrieved_docs['documents']:
    # Use either a unique ID or the content itself for uniqueness check
    unique_docs.add(doc[0])  # Replace 'id' with the unique identifier field name
print(f"Total Retrieved Documents: {len(retrieved_docs['documents'])}")
print(f"Unique Documents Retrieved: {len(unique_docs)}")

Total Retrieved Documents: 1
Unique Documents Retrieved: 1


In [87]:
unique_docs

{'to the 504 Loan.  \n \n2. Subordination of 504 Loan.  CDC agrees to make the 504 Loan to the Borrower , subject to SBA’s \napproval , and accept a junior and subordinate lien position in the Com mon Collateral upon the condition s that \nThird Party Lender executes this Agreement and disburses the  Third Party Loan  according to the terms \nrepresented to CDC and SBA .   \n \n3.   Accurate Information.  The Third Party Lender warrants and represents that all i nformation provided \nby the Third Party Lender  to CDC, including, without limitation, all information regarding the Borrower’s \nfinancial condition, is accurate to the best of its knowledge and that Third Party Lender has not withheld any \nmaterial informatio n.  Third Party Lender acknowledges that for purpose of this transaction, CDC is acting on \nbehalf of SBA, an agency in the United States Government, except that SBA accepts no liability or'}

In [88]:
# Extract the documents, embeddings, and metadata from the query results
retrieved_texts = [doc for doc in retrieved_docs['documents'][0]]
retrieved_embeddings = [embedding for embedding in retrieved_docs['embeddings'][0]]
retrieved_metadata = [meta for meta in retrieved_docs['metadatas'][0]]

In [89]:
# Step 8: Calculate the cosine similarity between the query and each of the retrieved document embeddings
similarities = [
    cosine_similarity([query_embedding], [embedding])[0][0] for embedding in retrieved_embeddings
]

In [90]:
# Step 9: Print the top 3 documents, their similarity scores, and page numbers
for i, (similarity, content, metadata) in enumerate(zip(similarities, retrieved_texts, retrieved_metadata)):
    # Extract the page number from the metadata
    page_number = metadata.get('page', None)
    
    # Print the document content, page number, and similarity score
    print(f"Top {i + 1} Document: Page {page_number+1}")
    print(f"Cosine Similarity: {similarity*100:.4f}")
    print(f"Content: {content}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

Top 1 Document: Page 2
Cosine Similarity: 95.5625
Content: to the 504 Loan.  
 
2. Subordination of 504 Loan.  CDC agrees to make the 504 Loan to the Borrower , subject to SBA’s 
approval , and accept a junior and subordinate lien position in the Com mon Collateral upon the condition s that 
Third Party Lender executes this Agreement and disburses the  Third Party Loan  according to the terms 
represented to CDC and SBA .   
 
3.   Accurate Information.  The Third Party Lender warrants and represents that all i nformation provided 
by the Third Party Lender  to CDC, including, without limitation, all information regarding the Borrower’s 
financial condition, is accurate to the best of its knowledge and that Third Party Lender has not withheld any 
material informatio n.  Third Party Lender acknowledges that for purpose of this transaction, CDC is acting on 
behalf of SBA, an agency in the United States Government, except that SBA accepts no liability or
################################

# Using Mistral <span style="color:red;">(PDF extraction and chunking not good)</span>

In [12]:
doc = fitz.open(pdf_file_path)

In [13]:
type(doc)

pymupdf.Document

In [14]:
metadata = []
current_chunk = ""
current_bbox = None  # To track the bounding box for the chunk
current_page_num = None

for page_num in range(len(doc)):
    page = doc[page_num]
    text_blocks = page.get_text("blocks")

    for block in text_blocks:
        text = block[4].strip()
        bbox = block[:4]  # Bounding box: (x0, y0, x1, y1)
        
        if text:  # Ensure the block contains text
            if len(current_chunk) + len(text) <= chunk_size:
                # Accumulate text in the current chunk
                if not current_chunk:  # Start a new chunk
                    current_bbox = bbox
                    current_page_num = page_num
                current_chunk += " " + text
            else:
                # Store the current chunk in metadata
                metadata.append({
                    'page_num': current_page_num,
                    'bbox': current_bbox,
                    'text': current_chunk.strip()
                })
                # Start a new chunk with the current block
                current_chunk = text
                current_bbox = bbox
                current_page_num = page_num

    # After processing all blocks on the page, check if there's a remaining chunk
    if current_chunk:
        metadata.append({
            'page_num': current_page_num,
            'bbox': current_bbox,
            'text': current_chunk.strip()
        })
        current_chunk = ""
        current_bbox = None

# After all pages are processed, ensure any remaining text is stored
if current_chunk:
    metadata.append({
        'page_num': current_page_num,
        'bbox': current_bbox,
        'text': current_chunk.strip()
    })


In [15]:
metadata

[{'page_num': 0,
  'bbox': (72.0, 736.0769653320312, 424.5137939453125, 756.4824829101562),
  'text': 'SBA Form 2287 (04-18) \n \n          1 \n Previous Editions Obsolete THIRD PARTY LENDER AGREEMENT THIS THIRD PARTY LENDER AGREEMENT (“Agreement") is dated this ______ day \nof______________, 20____, by and between ________________________________________, \n(“Third Party Lender”) whose address is ____________________________________________, \nand________________________________________________________, (“CDC”) whose address is \n_________________________________________________________________. RECITALS'},
 {'page_num': 0,
  'bbox': (71.98892211914062,
   324.6148681640625,
   542.1231689453125,
   453.1977844238281),
  'text': '1.  The Third Party Lender and CDC will provide separate loans to the Borrower and Operating Company, if \nany (collectively “Borrower”), according to the terms in the Authorization for Debenture Guarantee (SBA \n504 Loan), as amended (“Authorization”).  The 

In [16]:
# Initialize the embedding model using HuggingFaceEmbeddings with "mistral-7b"
embedding_model = HuggingFaceEmbeddings(model_name="bert-base-uncased")

C:\Users\jimmy\AppData\Local\Temp\ipykernel_17008\3851309224.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="bert-base-uncased")
C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


In [17]:
d = len(embedding_model.embed_query("test"))  # Determine the dimension

In [18]:
d

768

In [19]:
faiss_index = faiss.IndexFlatL2(d)
docstore = InMemoryDocstore()
vector_store = FAISS(index=faiss_index, docstore=docstore, index_to_docstore_id={}, embedding_function=embedding_model.embed_query)


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [20]:
# Step 4: Store each page's text and metadata in the FAISS vector database
for i, item in enumerate(metadata):
    text = item['text']
    embedding = embedding_model.embed_query(text)
    
    # Create a Document object with the text and metadata
    doc_obj = Document(page_content=text, metadata={'page_num': item['page_num'], 'bbox': item['bbox']})
    
    # Add the embedding and corresponding document to FAISS vector store
    vector_store.add_texts([text], [doc_obj], embeddings=[embedding])

ValidationError: 1 validation error for Document
metadata
  Input should be a valid dictionary [type=dict_type, input_value=Document(metadata={'page_...____________. RECITALS'), input_type=Document]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type

In [21]:
query

'What does this document say about Maturity and Balloon Payments?'

In [22]:
# Set up the retriever
retriever = vector_store.as_retriever()
retrieved_docs = retriever.get_relevant_documents(query)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_17008\1973722873.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [23]:
# Inspect what was retrieved
for i, d in enumerate(retrieved_docs):
    print(f"Document {i + 1}: {d.page_content}")
    print("#" * 100)  # Optional: To visually separate the chunks in the output

In [24]:
#retrieved_text = " ".join([d.page_content for d in retrieved_docs])
# consider only first retrived text
retrieved_text = retrieved_docs[0].page_content 

IndexError: list index out of range

In [25]:
retrieved_text

NameError: name 'retrieved_text' is not defined

* FAISS DB retrieves wrong text. Try with Chroma DB as below

In [26]:
# Use Chroma DB
# Initialize the OpenAI embedding model
embedding_model = OpenAIEmbeddings()
vector_store = Chroma(embedding_function=embedding_model, persist_directory="chroma_db")

# List to hold Document objects
documents = []

for item in metadata:
    text = item['text']
    bbox_str = str(item['bbox'])  # Convert bbox tuple to string
    doc_obj = Document(page_content=text, metadata={'page_num': item['page_num'], 'bbox': bbox_str})
    documents.append(doc_obj)



# Step 4: Add documents to the Chroma vector store
vector_store.add_documents(documents)
    
# Don't forget to persist the vector store
#vector_store.persist()

C:\Users\jimmy\AppData\Local\Temp\ipykernel_17008\3757482834.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()
C:\Users\jimmy\AppData\Local\Temp\ipykernel_17008\3757482834.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(embedding_function=embedding_model, persist_directory="chroma_db")


['9d4c2c83-bfba-4f40-8da0-7ed0090479ac',
 'ca4b042a-3f3b-4e33-9002-c9be803b3637',
 'de05dffc-ad0a-4476-bfb1-c1578b3ea30f',
 '947e522b-0b73-423b-9a5b-28db3b313d50',
 'ac3b50be-4d7c-4cc7-a476-4740118edbbf',
 'e89cbe4b-647c-43cc-b969-4f2dead14d88',
 '190da8f0-3e47-4825-b8ef-3e16e1e7857c',
 'cd360103-a52b-4068-afc6-1391014bd48c',
 '0c4f39d8-26cc-40b1-8137-1f24c288b6bf',
 '2d400fcc-59d8-4dc7-9d3a-737c7cf29e66',
 '670b8ba1-9524-496d-b8ff-e8e4c50c6a4f',
 'f343be43-21af-462b-9330-58927dbe35a0',
 'decf393b-ffa4-4eac-a8c3-abd6d9dd5866',
 '73c74ea2-8f66-400d-a001-c72accd072df',
 '6d8179ad-bf6b-4860-8a0d-a6bda802478f',
 '37a357e5-5c0b-49c9-ad49-6990524f096a',
 '6b8f2bca-4f9f-41d6-8ca7-b4f0bf0d4de3',
 '96e9b3f5-3d35-4ece-88ef-c23d58fcf482',
 'd669dd1d-d819-4b2d-9b06-d56b1f1038d3',
 'db5b46be-6aef-4306-be69-66eb7422f8e6',
 '5f976432-5f5f-4cdc-9722-fc32d01cf877',
 'a0f7c7af-20e6-4e07-81c6-67441f93bca9',
 '77eab1bf-ae0b-4e05-ae6f-6814485c7729',
 'd5b2b03b-a370-41a5-bb05-d8c0af00aead',
 'ce9625ef-7d75-

In [27]:
#query_embedding = embedding_model.embed_query(query)

In [28]:
results = vector_store.similarity_search_with_score(query, k=5)  # k defines how many results you want

In [29]:
# Display the results
for result, score in results:
    print(f"Score: {score}")
    print(f"Page Number: {result.metadata['page_num']}")
    print(f"Bounding Box: {result.metadata['bbox']}")
    print(f"Text: {result.page_content}\n")

Score: 0.3203049678362652
Page Number: 2
Bounding Box: (107.98898315429688, 196.81488037109375, 542.4618530273438, 262.1822509765625)
Text: exercise any cross-default, "deem at-risk," or any other provisions in documents evidencing the Third 
Party Loan or Third Party Lender Lien which allow Third Party Lender to make demand on the 
Third Party Loan prior to maturity unless the Third Party Loan is in material default.   
 
 
e. 
Maturity and Balloon Payments.   The Third Party Loan must have a term of at least 7 years (when the 504 loan is for a term of 10 years), or a term of at least10 years (when the 
504 loan is for 20 or 25 years).  If the Third Party Lender has made more than one loan, then an 
overall loan maturity must be calculated, taking into account the amounts and maturities of each 
loan.  Any balloon payment for the Third Party Loan must be clearly identified and disclosed to 
SBA and approved at application or subsequently approved by SBA. f. 
Reasonable Interest Rate. 

In [30]:
first_result = results[0][0]

In [31]:
bbox = first_result.metadata['bbox']
page_num = first_result.metadata['page_num']
first_text = first_result.page_content

In [32]:
bbox

'(107.98898315429688, 196.81488037109375, 542.4618530273438, 262.1822509765625)'

In [33]:
page_num

2

In [34]:
retrieved_text = first_text

In [35]:
retrieved_text

'exercise any cross-default, "deem at-risk," or any other provisions in documents evidencing the Third \nParty Loan or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party Loan is in material default.   \n \n \ne. \nMaturity and Balloon Payments.   The Third Party Loan must have a term of at least 7 years (when the 504 loan is for a term of 10 years), or a term of at least10 years (when the \n504 loan is for 20 or 25 years).  If the Third Party Lender has made more than one loan, then an \noverall loan maturity must be calculated, taking into account the amounts and maturities of each \nloan.  Any balloon payment for the Third Party Loan must be clearly identified and disclosed to \nSBA and approved at application or subsequently approved by SBA. f. \nReasonable Interest Rate.  The Third Party Loan has a reasonable interest rate'

In [36]:
input_text = f"""
Context: {retrieved_text}

Instruction: Answer the following question based only on the context provided. Do not use any external knowledge.

Question: {query}

Answer:
"""

In [37]:
# Prepare the input text for the LLM
#input_text = f"Context: {retrieved_text}\n\nQuestion: {query}\nAnswer:"

In [38]:
input_text

'\nContext: exercise any cross-default, "deem at-risk," or any other provisions in documents evidencing the Third \nParty Loan or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party Loan is in material default.   \n \n \ne. \nMaturity and Balloon Payments.   The Third Party Loan must have a term of at least 7 years (when the 504 loan is for a term of 10 years), or a term of at least10 years (when the \n504 loan is for 20 or 25 years).  If the Third Party Lender has made more than one loan, then an \noverall loan maturity must be calculated, taking into account the amounts and maturities of each \nloan.  Any balloon payment for the Third Party Loan must be clearly identified and disclosed to \nSBA and approved at application or subsequently approved by SBA. f. \nReasonable Interest Rate.  The Third Party Loan has a reasonable interest rate\n\nInstruction: Answer the following question based only on the 

In [39]:
# Use the LLM to generate an answer
answer = llm_mistral(input_text)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_17008\287618212.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = llm_mistral(input_text)


In [40]:
answer

'\nAccording to the context, the Third Party Loan must have a term of at least 7 years when the 504 loan is for a term of 10 years, or a term of at least 10 years when the 504 loan is for 25 or 20 years. The document also specifies that any balloon payment for the Third Party Loan must be clearly identified and disclosed to SBA and approved at application or subsequently approved by SBA.'

In [41]:
highlighted_pdf_path = 'doc_outputs/highlighted_file_faiss.pdf'

In [42]:
# Generate an embedding for the LLM's response
response_embedding = embedding_model.embed_query(answer)

In [43]:
# Perform cosine similarity search using LangChain's FAISS index
similar_docs = vector_store.similarity_search_with_score(answer, k=1)

In [44]:
# Retrieve the best-matching section
best_match = similar_docs[0][0].metadata
best_text = similar_docs[0][0].page_content
best_score = similar_docs[0][1]

In [45]:
best_match

{'bbox': '(107.98898315429688, 196.81488037109375, 542.4618530273438, 262.1822509765625)',
 'page_num': 2}

In [46]:
type(best_match)

dict

In [47]:
# Access the nested metadata dictionary
#metadata = best_match.get('metadata', {})
metadata = best_match

# Retrieve page_num and bbox from the metadata
page_num = metadata.get('page_num')
bbox = metadata.get('bbox')

In [48]:
page_num

2

In [49]:
bbox

'(107.98898315429688, 196.81488037109375, 542.4618530273438, 262.1822509765625)'

In [50]:
best_text

'exercise any cross-default, "deem at-risk," or any other provisions in documents evidencing the Third \nParty Loan or Third Party Lender Lien which allow Third Party Lender to make demand on the \nThird Party Loan prior to maturity unless the Third Party Loan is in material default.   \n \n \ne. \nMaturity and Balloon Payments.   The Third Party Loan must have a term of at least 7 years (when the 504 loan is for a term of 10 years), or a term of at least10 years (when the \n504 loan is for 20 or 25 years).  If the Third Party Lender has made more than one loan, then an \noverall loan maturity must be calculated, taking into account the amounts and maturities of each \nloan.  Any balloon payment for the Third Party Loan must be clearly identified and disclosed to \nSBA and approved at application or subsequently approved by SBA. f. \nReasonable Interest Rate.  The Third Party Loan has a reasonable interest rate'

In [51]:
if page_num is None or bbox is None:
    print("Metadata is missing 'page_num' or 'bbox'.")
else:
    print(f"Best Match Found: {best_text} with score {best_score} on page {page_num}")

Best Match Found: exercise any cross-default, "deem at-risk," or any other provisions in documents evidencing the Third 
Party Loan or Third Party Lender Lien which allow Third Party Lender to make demand on the 
Third Party Loan prior to maturity unless the Third Party Loan is in material default.   
 
 
e. 
Maturity and Balloon Payments.   The Third Party Loan must have a term of at least 7 years (when the 504 loan is for a term of 10 years), or a term of at least10 years (when the 
504 loan is for 20 or 25 years).  If the Third Party Lender has made more than one loan, then an 
overall loan maturity must be calculated, taking into account the amounts and maturities of each 
loan.  Any balloon payment for the Third Party Loan must be clearly identified and disclosed to 
SBA and approved at application or subsequently approved by SBA. f. 
Reasonable Interest Rate.  The Third Party Loan has a reasonable interest rate with score 0.1153896453260796 on page 2


In [52]:
# Step 7: Highlight the response in the PDF document
page = doc.load_page(page_num)    

In [53]:
quads = [(bbox[0], bbox[1], bbox[2], bbox[1], bbox[2], bbox[3], bbox[0], bbox[3])]

In [54]:
page

page 2 of doc_inputs/Form_2287.pdf

In [55]:
page.add_highlight_annot(bbox)

ValueError: bad quads entry

In [56]:
type(page)

pymupdf.Page

In [57]:
# Save the highlighted PDF
doc.save(highlighted_pdf_path)

# Conclusion

<ol style="font-size: 20px;">
  <li>PyPDFLoader and ChromaDB extracts the right content. Both Mistral and GPT works well as LLM model. For highlighting text cosine similarity serach works fine </li>
  <li>FAISSDB doesn't work well to get right context. Highlighting based on fitz based metadata gives error.</li>  
</ol>